In [ ]:
import lime
import lime.lime_tabular
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn import metrics
import xgboost as xgb
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


def CodeprocessGerman(data):
    data['predict'] = LabelEncoder().fit_transform(data['predict'])
    # 这里采用独热编码
    # 将字符串编码为无关的数字
    data['Status of existing checking account'] = LabelEncoder().fit_transform(data['Status of existing checking account'])
    data = pd.get_dummies(data, columns=['Status of existing checking account'])
    data['Credit history'] = LabelEncoder().fit_transform(data['Credit history'])
    data = pd.get_dummies(data, columns=['Credit history'])
    data['Purpose'] = LabelEncoder().fit_transform(data['Purpose'])
    data = pd.get_dummies(data, columns=['Purpose'])
    data['Savings account bonds'] = LabelEncoder().fit_transform(data['Savings account bonds'])
    data = pd.get_dummies(data, columns=['Savings account bonds'])
    data['Present employment since'] = LabelEncoder().fit_transform(data['Present employment since'])
    data = pd.get_dummies(data, columns=['Present employment since'])
    data['Personal status and sex'] = LabelEncoder().fit_transform(data['Personal status and sex'])
    data = pd.get_dummies(data, columns=['Personal status and sex'])
    data['Other debtors  guarantors'] = LabelEncoder().fit_transform(data['Other debtors  guarantors'])
    data = pd.get_dummies(data, columns=['Other debtors  guarantors'])
    data['Property'] = LabelEncoder().fit_transform(data['Property'])
    data = pd.get_dummies(data, columns=['Property'])
    data['Other installment plans'] = LabelEncoder().fit_transform(data['Other installment plans'])
    data = pd.get_dummies(data, columns=['Other installment plans'])
    data['Housing'] = LabelEncoder().fit_transform(data['Housing'])
    data = pd.get_dummies(data, columns=['Housing'])
    data['highly qualified employee officer'] = LabelEncoder().fit_transform(data['highly qualified employee officer'])
    data = pd.get_dummies(data, columns=['highly qualified employee officer'])
    data['Telephone'] = LabelEncoder().fit_transform(data['Telephone'])
    data = pd.get_dummies(data, columns=['Telephone'])
    data['foreign worker'] = LabelEncoder().fit_transform(data['foreign worker'])
    data = pd.get_dummies(data, columns=['foreign worker'])
    return data
data_adult = pd.read_csv('../../data/adult1.csv')
data_german = pd.read_csv('../../data/german.csv')

data_adult = CodeprocessAdult(data_adult)
data_german = CodeprocessGerman(data_german)
Y_adult = data_adult["predict"]
X_adult = data_adult.drop('predict', axis=1)
Y_german = data_german["predict"]
X_german = data_german.drop('predict', axis=1)

X_train_adult, X_test_adult, Y_train_adult, Y_test_adult = train_test_split(X_adult, Y_adult)
X_train_german, X_test_german, Y_train_german, Y_test_german = train_test_split(X_german, Y_german)


# def randomForestAdultTrain(X_train,Y_train,X_test,Y_test):
#     # random forest
#     # 训练随机森林
#     randomForest = RandomForestClassifier(n_estimators=800,
#                                           criterion='gini',
#                                           max_depth=None,
#                                           min_samples_split=2,
#                                           min_samples_leaf=1,
#                                           min_weight_fraction_leaf=0.0,
#                                           max_features='auto',
#                                           max_leaf_nodes=None,
#                                           min_impurity_decrease=0.0,
#                                           min_impurity_split=None,
#                                           bootstrap=True,
#                                           oob_score=True,
#                                           n_jobs=1,
#                                           random_state=None,
#                                           verbose=0,
#                                           warm_start=False,
#                                           class_weight=None)
#     randomForest.fit(X_train, Y_train)
#     Predictions = randomForest.predict(X_test)
#     scoreRandomForest = randomForest.score(X_train, Y_train)

#     print("adult randomforest prediction:")
#     print(classification_report(Y_test, Predictions))
#     # print(scoreRandomForest)
#     return randomForest

# randomForestAdult = randomForestAdultTrain(X_train_adult,Y_train_adult,X_test_adult,Y_test_adult)
# joblib.dump(randomForestAdult, 'D:/Code/PycharmProjects/Interpretability/interpretTest/model/randomforestadult.pkl')
randomForestAdult = joblib.load('../model/randomforestadult.pkl')


def Lime_randomforest_adult(randomforest,X_test,X_train,data):
    columns = data.columns
    datacolumns = []
    for this_column in columns:
        if this_column != "predict":
            datacolumns.append(this_column)
    classnames=[' <=50K', ' >50K']
    data = data.values
    X_train = X_train.values
    explainer = lime.lime_tabular.LimeTabularExplainer(X_train, class_names=classnames, feature_names=datacolumns,
                                                       discretize_continuous=True)
    # i = np.random.randint(0, X_test.shape[0])
    # exp = explainer.explain_instance(X_test[1], xgboost.predict_proba, num_features=2, top_labels=1)
    # exp.show_in_notebook(show_table=True, show_all=False)

    i = np.random.randint(0, X_test.shape[0])

    example = X_test.iloc[i]

    file = open('../exampleNew/randomforest_adult.txt', mode='w')
    for i, v in example.iteritems():
        file.write(str(i))
        file.write(" ")
        file.write(str(v))
        file.write("\n")
    file.close()
    exp = explainer.explain_instance(example, randomforest.predict_proba,num_features=X_train.shape[1])
    return exp
    
exp = Lime_randomforest_adult(randomForestAdult,X_test_adult,X_train_adult,data_adult)

exp.show_in_notebook(show_all=False)